### Manipulation of PeaskDB de novo-assisted database search results of Trocas 8 (low water, November 2019) non incubation samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 675) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [1]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/688_Trocas8_June2021_PEAKS_148/

/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/688_Trocas8_June2021_PEAKS_148


In [4]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [5]:
ls 

 688_Trocas8_June2021_PEAKS_148_peptide.csv     'de novo only peptides.csv'
 688_Trocas8_June2021_PEAKS_148_proteins.csv     peptides_1_1_0.mzid
 688_Trocas8_June2021_PEAKS_148_proteins.fasta   protein-peptides.csv
'DB search psm.csv'


In [6]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup688 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/688_Trocas8_June2021_PEAKS_148/688_Trocas8_June2021_PEAKS_148_peptide.csv")

# remove redundant rows
peaksdb688 = pd.DataFrame.drop_duplicates(peaksdbdup688)

print(peaksdb688.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source_File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb688.columns = columns

#remmove # spec and accession columns because they mess parsing up

del peaksdb688['#Spec']
del peaksdb688['Accession']
del peaksdb688['PTM']
del peaksdb688['AScore']

mean_length = peaksdb688['Length'].mean()
print(mean_length)

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup688))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb688))

#look at the dataframe
peaksdb688.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area Sample 15', 'Fraction', 'Scan', 'Source File', '#Spec',
       '#Spec Sample 15', 'Accession', 'PTM', 'AScore'],
      dtype='object')
9.274193548387096
# redundant peaksdb peptides in combined dataframe 62
# nonredundant peaksdb peptides in combined dataframe 62


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source_File
0,GSGGGSSGGSIGGR,53.59,1091.4955,14,-0.9,546.7545,17.65,9040.0,15,2680,20210625_Trocas8_688_SMCP64_surface_2_DDA_120m...
1,LSSPATLNSR,49.08,1044.5564,10,-1.3,523.2848,45.33,9880000.0,15,7400,20210625_Trocas8_688_SMCP64_surface_2_DDA_120m...
2,LVNELTEFAK,42.99,1162.6233,10,-0.9,582.3184,86.14,42500.0,15,14440,20210625_Trocas8_688_SMCP64_surface_2_DDA_120m...
3,SC(+57.02)AAAGTEC(+57.02)L,38.71,1038.4110,10,-0.7,520.2124,49.68,81100.0,15,8234,20210625_Trocas8_688_SMCP64_surface_2_DDA_120m...
4,LATVISPR,38.43,855.5178,8,-1.5,428.7655,59.91,6470000.0,15,9983,20210625_Trocas8_688_SMCP64_surface_2_DDA_120m...


In [7]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb688['A'] = peaksdb688['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb688['C'] = peaksdb688['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb688['D'] = peaksdb688['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb688['E'] = peaksdb688['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb688['F'] = peaksdb688['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb688['G'] = peaksdb688['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb688['H'] = peaksdb688['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb688 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb688['I'] = peaksdb688['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb688['K'] = peaksdb688['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb688['L'] = peaksdb688['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb688['M'] = peaksdb688['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb688['N'] = peaksdb688['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb688['P'] = peaksdb688['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb688['Q'] = peaksdb688['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb688['R'] = peaksdb688['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb688['S'] = peaksdb688['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb688['T'] = peaksdb688['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb688['V'] = peaksdb688['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb688['W'] = peaksdb688['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb688['Y'] = peaksdb688['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb688['c-carb'] = peaksdb688['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb688['m-oxid'] = peaksdb688['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb688['n-deam'] = peaksdb688['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a lamba function to enumerate the # of deamidated Q's in each peptide
peaksdb688['q-deam'] = peaksdb688['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb688['stripped_peptide'] = peaksdb688['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb688['stripped_length'] = peaksdb688['stripped_peptide'].apply(len)

peaksdb688['NAAF_num.'] = peaksdb688['Area'] / peaksdb688['stripped_length']

# total the number of modifications in sequence
peaksdb688['ptm-total'] = peaksdb688['c-carb'] + peaksdb688['m-oxid'] + peaksdb688['n-deam'] + peaksdb688['q-deam']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb688['stripped_IL']= peaksdb688['stripped_peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb688.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/680_SMCP_surf_GF75_PDB/688_SMCP_surf_GF75_PDB.csv")

# check out the results
peaksdb688.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,NAAF_num.,ptm-total,stripped_IL
0,GSGGGSSGGSIGGR,53.59,1091.4955,14,-0.9,546.7545,17.65,9040.0,15,2680,...,0,0,0,0,0,GSGGGSSGGSIGGR,14,645.714286,0,GSGGGSSGGSLGGR
1,LSSPATLNSR,49.08,1044.5564,10,-1.3,523.2848,45.33,9880000.0,15,7400,...,0,0,0,0,0,LSSPATLNSR,10,988000.000000,0,LSSPATLNSR
2,LVNELTEFAK,42.99,1162.6233,10,-0.9,582.3184,86.14,42500.0,15,14440,...,0,0,0,0,0,LVNELTEFAK,10,4250.000000,0,LVNELTEFAK
3,SC(+57.02)AAAGTEC(+57.02)L,38.71,1038.4110,10,-0.7,520.2124,49.68,81100.0,15,8234,...,0,2,0,0,0,SCL,3,27033.333333,2,SCL
4,LATVISPR,38.43,855.5178,8,-1.5,428.7655,59.91,6470000.0,15,9983,...,0,0,0,0,0,LATVISPR,8,808750.000000,0,LATVLSPR


In [8]:
# keep only stripped peptide I/L and NAAF
db_688 = peaksdb688[['stripped_IL', 'Area', 'NAAF_num.']]

db_688.set_index('stripped_IL')

# write modified dataframe to new txt file
db_688.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/I-L_NAAFs/688_SMCP_surf_GF75_PDB_ILnaafs.csv")

db_688.head()

,stripped_IL,Area,NAAF_num.
0,GSGGGSSGGSLGGR,9040.0,645.714286
1,LSSPATLNSR,9880000.0,988000.000000
2,LVNELTEFAK,42500.0,4250.000000
3,SCL,81100.0,27033.333333
4,LATVLSPR,6470000.0,808750.000000


### Exporting txt files of stripped peptides at confidence cutoffs:

In [9]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep688moddup = peaksdb688[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep688dup = peaksdb688[["stripped_peptide"]]

# deduplicate both of these lists
pep688mod = pep688moddup.drop_duplicates()
pep688 = pep688dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 688:", len(pep688moddup))
print("Deduplicated modified PeaksDB peptides in 688:", len(pep688mod))
print("Total stripped PeaksDB peptides in 688:", len(pep688dup))
print("Deduplicated stripped PeaksDB peptides in 688:", len(pep688))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep688.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/680_SMCP_surf_GF75_PDB/688_SMCP_surf_GF75_PDB_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/680_SMCP_surf_GF75_PDB/688_SMCP_surf_GF75_PDB_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/680_SMCP_surf_GF75_PDB/688_SMCP_surf_GF75_PDB_stripped_peptides.fas

# look at the stripped peptides
pep688.head()

Total modified PeaksDB peptides in 688: 62
Deduplicated modified PeaksDB peptides in 688: 62
Total stripped PeaksDB peptides in 688: 62
Deduplicated stripped PeaksDB peptides in 688: 62


,stripped_peptide
0,GSGGGSSGGSIGGR
1,LSSPATLNSR
2,LVNELTEFAK
3,SCL
4,LATVISPR


## NAAF correction and exporting files with AA and PTM totals:

In [10]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb688['A'].sum(),
        'C': peaksdb688['C'].sum(),
        'D': peaksdb688['D'].sum(),
        'E': peaksdb688['E'].sum(),
        'F': peaksdb688['F'].sum(),
        'G': peaksdb688['G'].sum(),
        'H': peaksdb688['H'].sum(),
        'I': peaksdb688['I'].sum(),
        'K': peaksdb688['K'].sum(),
        'L': peaksdb688['L'].sum(),
        'M': peaksdb688['M'].sum(),
        'N': peaksdb688['N'].sum(),
        'P': peaksdb688['P'].sum(),
        'Q': peaksdb688['Q'].sum(),
        'R': peaksdb688['R'].sum(),
        'S': peaksdb688['S'].sum(),
        'T': peaksdb688['T'].sum(),
        'V': peaksdb688['V'].sum(),
        'W': peaksdb688['W'].sum(),
        'Y': peaksdb688['Y'].sum(),
        'c-carb': peaksdb688['c-carb'].sum(),
        'm-oxid': peaksdb688['m-oxid'].sum(),
        'n-deam': peaksdb688['n-deam'].sum(),
        'q-deam': peaksdb688['q-deam'].sum(),
        'Total area': peaksdb688['Area'].sum(),
        'Total length': peaksdb688['stripped_length'].sum()
       }

totalpeaksdb688 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 
                                              'n-deam', 'q-deam', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb688['% C w/ carb'] = totalpeaksdb688['c-carb'] / totalpeaksdb688['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb688['% M w/ oxid'] = totalpeaksdb688['m-oxid'] / totalpeaksdb688['M'] 

# calculate percentage of N's that are deamidated
totalpeaksdb688['% N w/ deam'] = totalpeaksdb688['n-deam'] / totalpeaksdb688['N'] 

# calculate percentage of N's that are deamidated
totalpeaksdb688['% Q w/ deam'] = totalpeaksdb688['q-deam'] / totalpeaksdb688['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb688['NAAF denom.'] = totalpeaksdb688['Total area'] / totalpeaksdb688['Total length']

# write modified dataframe to new txt file
totalpeaksdb688.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/680_SMCP_surf_GF75_PDB/688_SMCP_surf_GF75_PDB_totals.csv")

totalpeaksdb688.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,38,2,15,20,21,40,13,35,44,76,...,4,2,0,64108507.0,568,1.0,0.4,0.1,0.0,112867.089789


In [11]:
# use the calculated NAAF factor (in totalpeaksdb688 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 112867.089789

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb688['NAAF factor'] = (peaksdb688['NAAF_num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksdb688_NAAF = peaksdb688[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'n-deam', 'q-deam']].copy()

# multiply the NAAF20 factor SMCP the AAs to normalize its abundance SMCP peak area and peptide length

peaksdb688_NAAF['A-NAAF20'] = peaksdb688_NAAF['A'] * peaksdb688['NAAF factor']
peaksdb688_NAAF['C-NAAF20'] = peaksdb688_NAAF['C'] * peaksdb688['NAAF factor']
peaksdb688_NAAF['D-NAAF20'] = peaksdb688_NAAF['D'] * peaksdb688['NAAF factor']
peaksdb688_NAAF['E-NAAF20'] = peaksdb688_NAAF['E'] * peaksdb688['NAAF factor']
peaksdb688_NAAF['F-NAAF20'] = peaksdb688_NAAF['F'] * peaksdb688['NAAF factor']
peaksdb688_NAAF['G-NAAF20'] = peaksdb688_NAAF['G'] * peaksdb688['NAAF factor']
peaksdb688_NAAF['H-NAAF20'] = peaksdb688_NAAF['H'] * peaksdb688['NAAF factor']
peaksdb688_NAAF['I-NAAF20'] = peaksdb688_NAAF['I'] * peaksdb688['NAAF factor']
peaksdb688_NAAF['K-NAAF20'] = peaksdb688_NAAF['K'] * peaksdb688['NAAF factor']
peaksdb688_NAAF['L-NAAF20'] = peaksdb688_NAAF['L'] * peaksdb688['NAAF factor']
peaksdb688_NAAF['M-NAAF20'] = peaksdb688_NAAF['M'] * peaksdb688['NAAF factor']
peaksdb688_NAAF['N-NAAF20'] = peaksdb688_NAAF['N'] * peaksdb688['NAAF factor']
peaksdb688_NAAF['P-NAAF20'] = peaksdb688_NAAF['P'] * peaksdb688['NAAF factor']
peaksdb688_NAAF['Q-NAAF20'] = peaksdb688_NAAF['Q'] * peaksdb688['NAAF factor']
peaksdb688_NAAF['R-NAAF20'] = peaksdb688_NAAF['R'] * peaksdb688['NAAF factor']
peaksdb688_NAAF['S-NAAF20'] = peaksdb688_NAAF['S'] * peaksdb688['NAAF factor']
peaksdb688_NAAF['T-NAAF20'] = peaksdb688_NAAF['T'] * peaksdb688['NAAF factor']
peaksdb688_NAAF['V-NAAF20'] = peaksdb688_NAAF['V'] * peaksdb688['NAAF factor']
peaksdb688_NAAF['W-NAAF20'] = peaksdb688_NAAF['W'] * peaksdb688['NAAF factor']
peaksdb688_NAAF['Y-NAAF20'] = peaksdb688_NAAF['Y'] * peaksdb688['NAAF factor']

# multiply the NAAF20 factor SMCP the PTMs normalize its abundance SMCP peak area and peptide length

peaksdb688_NAAF['ccarb-NAAF20'] = peaksdb688_NAAF['c-carb'] * peaksdb688_NAAF['NAAF factor']
peaksdb688_NAAF['moxid-NAAF20'] = peaksdb688_NAAF['m-oxid'] * peaksdb688_NAAF['NAAF factor']
peaksdb688_NAAF['ndeam-NAAF20'] = peaksdb688_NAAF['n-deam'] * peaksdb688_NAAF['NAAF factor']
peaksdb688_NAAF['qdeam-NAAF20'] = peaksdb688_NAAF['q-deam'] * peaksdb688_NAAF['NAAF factor']


# write the dataframe to a new csv
peaksdb688_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/680_SMCP_surf_GF75_PDB/688_SMCP_surf_GF75_PDB_naaf.csv")

peaksdb688_NAAF.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,I,...,R-NAAF20,S-NAAF20,T-NAAF20,V-NAAF20,W-NAAF20,Y-NAAF20,ccarb-NAAF20,moxid-NAAF20,ndeam-NAAF20,qdeam-NAAF20
0,GSGGGSSGGSIGGR,0.005721,0,0,0,0,0,8,0,1,...,0.005721,0.022884,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0
1,LSSPATLNSR,8.753659,1,0,0,0,0,0,0,0,...,8.753659,26.260977,8.753659,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0
2,LVNELTEFAK,0.037655,1,0,0,2,1,0,0,0,...,0.000000,0.000000,0.037655,0.037655,0.0,0.0,0.00000,0.0,0.0,0.0
3,SCL,0.239515,3,2,0,1,0,1,0,0,...,0.000000,0.239515,0.239515,0.000000,0.0,0.0,0.47903,0.0,0.0,0.0
4,LATVISPR,7.165508,1,0,0,0,0,0,0,1,...,7.165508,7.165508,7.165508,7.165508,0.0,0.0,0.00000,0.0,0.0,0.0


In [12]:
# made a new dataframe that contains the sums of NAAF normalized AAs for peaksdb688 peaksdb results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb688_NAAF['NAAF factor'].sum(),
        'A': peaksdb688_NAAF['A-NAAF20'].sum(),
        'C': peaksdb688_NAAF['C-NAAF20'].sum(),
        'D': peaksdb688_NAAF['D-NAAF20'].sum(),
        'E': peaksdb688_NAAF['E-NAAF20'].sum(),
        'F': peaksdb688_NAAF['F-NAAF20'].sum(),
        'G': peaksdb688_NAAF['G-NAAF20'].sum(),
        'H': peaksdb688_NAAF['H-NAAF20'].sum(),
        'I': peaksdb688_NAAF['I-NAAF20'].sum(),
        'K': peaksdb688_NAAF['K-NAAF20'].sum(),
        'L': peaksdb688_NAAF['L-NAAF20'].sum(),
        'M': peaksdb688_NAAF['M-NAAF20'].sum(),
        'N': peaksdb688_NAAF['N-NAAF20'].sum(),
        'P': peaksdb688_NAAF['P-NAAF20'].sum(),
        'Q': peaksdb688_NAAF['Q-NAAF20'].sum(),
        'R': peaksdb688_NAAF['R-NAAF20'].sum(),
        'S': peaksdb688_NAAF['S-NAAF20'].sum(),
        'T': peaksdb688_NAAF['T-NAAF20'].sum(),
        'V': peaksdb688_NAAF['V-NAAF20'].sum(),
        'W': peaksdb688_NAAF['W-NAAF20'].sum(),
        'Y': peaksdb688_NAAF['Y-NAAF20'].sum(),
        'c-carb': peaksdb688_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb688_NAAF['moxid-NAAF20'].sum(),
        'n-deam': peaksdb688_NAAF['ndeam-NAAF20'].sum(),
        'q-deam': peaksdb688_NAAF['qdeam-NAAF20'].sum()
       }

totalpeaksdb688_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb688_NAAF['% C w/ carb.'] = totalpeaksdb688_NAAF['c-carb'] / totalpeaksdb688_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb688_NAAF['% M w/ oxid'] = totalpeaksdb688_NAAF['m-oxid'] / totalpeaksdb688_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb688_NAAF['% N w/ deam'] = totalpeaksdb688_NAAF['n-deam'] / totalpeaksdb688_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb688_NAAF['% Q w/ deam'] = totalpeaksdb688_NAAF['q-deam'] / totalpeaksdb688_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb688_NAAF['NAAF check'] = totalpeaksdb688_NAAF['NAAF'] / 112867.089789

# write modified dataframe to new txt file, same name + totals
totalpeaksdb688_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/680_SMCP_surf_GF75_PDB/688_SMCP_surf_GF75_PDB_naaf_totals.csv")

totalpeaksdb688_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,68.828874,43.796609,0.47903,0.622929,0.958484,0.602916,1.827155,2.150136,24.39497,3.691541,...,0.374193,0.47903,0.046043,0.097138,0.0,1.0,0.287843,0.010019,0.0,0.00061
